# The Decomposition Pattern - Breaking Down Big Problems

Hello everyone, let's now explore the **Decomposition Pattern**, which can be very useful in tackling more complex problems.

The core idea is simple but powerful: **Don't solve one big problem. Solve many small problems.**

We break down a complex task into a sequence of smaller, focused sub-tasks. Each sub-task is handled by its own dedicated prompt, creating a "pipeline" where the output of one step becomes the input for the next.

This notebook will demonstrate this by building a three-step pipeline for a common developer workflow:

1. **Step 1:** Identify bugs in a piece of code.
2. **Step 2:** Refactor the code to fix the identified bugs.
3. **Step 3:** Generate unit tests for the newly refactored code.

## The Task and Helper Functions

First, let's define our helper functions and the initial "buggy" code that we will process through our pipeline.

In [ ]:
import litellm
import inspect
import json
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    if "gpt-5" in model:
        kwargs["max_completion_tokens"] = max_tokens
    else:
        kwargs["max_tokens"] = max_tokens
        
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        **kwargs
    )

    return response.choices[0].message.content

def get_average_score(student_records):
    total_score = 0
    for record in student_records:
        total_score += record['score']

    return total_score / len(student_records)

CODE_TO_PROCESS = inspect.getsource(get_average_score)

print("Setup complete. Helper functions and code context are ready.")

## Bug Identification

Our first prompt is highly focused. Its only job is to analyze the code and identify potential bugs. We'll ask for the output in JSON format so we can easily parse it for the next step.

In [ ]:
prompt_bug_identification = [
    {
        "role": "system",
        "content": dedent("""
        You are a master code auditor.
        Your sole task is to analyze Python code and identify potential bugs, errors, and edge-cases.

        Do NOT suggest fixes.

        Output your findings as a JSON object with the following schema:
        
        {
            "bugs": list(str)
        }
        """)
    },
    {
        "role": "user",
        "content": dedent(f"""
        Please analyze the following Python code:

        ```python
        {CODE_TO_PROCESS}
        ```
        """)
    }
]

print("--- Step 1: Identifying bugs ---")
identified_bugs_json = get_completion(
    prompt_bug_identification,
    response_format={"type": "json_object"}
)

if identified_bugs_json:
    print("Identified issues:")
    print(identified_bugs_json)
    identified_bugs = json.loads(identified_bugs_json).get("bugs", [])
else:
    identified_bugs = []

## Refactoring

Now, we create our second prompt. Its context includes **both** the original code and the list of bugs we identified in Step 1. The task is to fix these specific issues.

In [ ]:
prompt_refactoring = [
    {
        "role": "system",
        "content": dedent("""
        You are a senior Python developer.
        Your task is to refactor code to fix identified issues and improve its quality and robustness.
        """)
    },
    {
        "role": "user",
        "content": dedent(f"""
        Please refactor the following Python code to fix the idenfied issues.
        Add a docstring and type hints.

        ## Original code
        ```python
        {CODE_TO_PROCESS}
        ```

        ## Identified issues
        {"* ".join(identified_bugs)}

        ## Response instructions
        * Your response ahould be only the complete, refactored Python code.
        * Do not output any triple backticks nor code blocks, just directly the code.
        * Output only valid Python code.
        """)
    }
]

print("--- Step 2: Refactoring the code ---")
refactored_code = get_completion(prompt_refactoring)

if refactored_code:
    print("Refactored code:")
    print(refactored_code)

## Test Generation

Finally, our third prompt takes the **refactored code from Step 2** as its context. Its single, focused goal is to generate a comprehensive suite of unit tests.

In [ ]:
prompt_generate_unit_tests = [
    {
        "role": "system",
        "content": dedent("""
        You are an expert QA engineer specializing in Python.
        Your task is to write a comprehensive suite of unit tests for a given function.
        You always use the pytest framework.
        """)
    },
    {
        "role": "user",
        "content": dedent(f"""
        Please write unit tests for the following refactored Python function.

        The tests should cover the main functionality and edge-cases.
        
        Output the content in XML strictly following the structure in the Output section.
        Do not output any additional content outside of the XML tags.

        ## Refactored code
        {refactored_code}

        ## Output
        <preamble>
            Add your preamble here.
        </preamble>
        <reasoning>
            Add your reasoning here.
        </reasoning>
        <edge_cases>
            Add all identified edge-cases here.
        </edge_cases>
        <python_code>
            Add the test code here.
        </python_code>
        <conclusion>
            Add the conclusion here
        </conclusion>
        """)
    }
]

print("--- Step 3: Generating unit tests ---")
unit_tests = get_completion(prompt_generate_unit_tests, max_tokens=2000)

if unit_tests:
    print("Generated tests:")
    print(unit_tests)